### **1. Import Library**

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# Mengatur agar angka desimal ditampilkan rapi (bukan notasi ilmiah)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# Mengabaikan warning dari geopandas agar output bersih
warnings.filterwarnings('ignore', 'UserWarning', module='geopandas')

print("Libraries diimport.")

Libraries diimport.


### **2. Path**

In [2]:
path_data_clean = "../data_clean/"
path_maps = "../maps/"
path_visualisasi = "../visualisasi_EDA/" # Folder untuk menyimpan hasil gambar & tabel

# Buat folder visualisasi klo belum ada
os.makedirs(path_visualisasi, exist_ok=True)

print(f"Data diambil dari: {path_data_clean}")
print(f"Peta diambil dari: {path_maps}")
print(f"Hasil EDA akan disimpan di: {path_visualisasi}")

Data diambil dari: ../data_clean/
Peta diambil dari: ../maps/
Hasil EDA akan disimpan di: ../visualisasi_EDA/


### **3. Load Data Bersih**

In [3]:
file_data_bersih = os.path.join(path_data_clean, "dataset_final_2021-2023.csv")

try:
    df = pd.read_csv(file_data_bersih)
    print(f"Berhasil memuat data: {file_data_bersih}")
    print(f"Dimensi data: {df.shape}")
    print(df.head())
except FileNotFoundError:
    print(f"ERROR: File {file_data_bersih} tidak ditemukan.")
    print("Jalankan '01_preprocessing.ipynb' terlebih dahulu!")
    raise

# Pastikan tipe data
cols_to_numeric = ['P1', 'UHH', 'HLS', 'RLS', 'Pengeluaran', 'TPT', 'Kepadatan']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Pisahkan hanya kolom numerik untuk analisis
df_numeric = df[cols_to_numeric]

Berhasil memuat data: ../data_clean/dataset_final_2021-2023.csv
Dimensi data: (105, 9)
                  Wilayah  Tahun   P1   UHH   HLS  RLS  Pengeluaran  TPT  \
0       Kabupaten Cilacap   2021 1.48 73.90 12.63 7.09        10534 9.97   
1      Kabupaten Banyumas   2021 2.35 73.80 13.03 7.63        11546 6.05   
2   Kabupaten Purbalingga   2021 2.10 73.21 12.00 7.25        10032 6.05   
3  Kabupaten Banjarnegara   2021 2.97 74.28 11.63 6.75         9407 5.86   
4       Kabupaten Kebumen   2021 3.24 73.55 13.35 7.55         9028 6.03   

   Kepadatan  
0        924  
1       1340  
2       1487  
3       1003  
4       1124  


### **4. Statistik Deskriptif**

In [4]:
deskriptif = df_numeric.describe().T
print(deskriptif)

# Simpan ke CSV
# deskriptif.to_csv(os.path.join(path_visualisasi, "statistik_deskriptif.csv"))
# print("\nFile tersimpan: statistik_deskriptif.csv")

             count     mean     std     min      25%      50%      75%  \
P1          105.00     1.62    0.69    0.47     1.03     1.57     1.95   
UHH         105.00    75.12    1.81   69.54    74.03    74.87    76.39   
HLS         105.00    13.01    0.91   11.63    12.44    12.89    13.35   
RLS         105.00     8.13    1.27    6.22     7.26     7.79     8.79   
Pengeluaran 105.00 11565.74 1814.00 8573.00 10277.00 11110.00 12522.00   
TPT         105.00     5.36    1.95    1.76     4.05     5.03     6.35   
Kepadatan   105.00  2113.94 2413.25  461.00   930.00  1180.00  1846.00   

                 max  
P1              3.41  
UHH            77.93  
HLS            15.55  
RLS            11.24  
Pengeluaran 16650.00  
TPT             9.97  
Kepadatan   11878.00  


### **5. Visualisasi Distribusi (Histogram)**

In [ ]:
print("Membuat Histogram Distribusi Data...")

# Layout grid (3 kolom)
n_cols = 3
n_rows = math.ceil(len(df_numeric.columns) / n_cols)

colors = px.colors.qualitative.Bold

fig = make_subplots(rows=n_rows, cols=n_cols, 
                    subplot_titles=df_numeric.columns, # Judul tiap grafik
                    vertical_spacing=0.1)

for i, column in enumerate(df_numeric.columns):
    row = (i // n_cols) + 1
    col = (i % n_cols) + 1
    
    color = colors[i % len(colors)]
    
    # Tambahkan Histogram untuk variabel
    fig.add_trace(
        go.Histogram(
            x=df_numeric[column], 
            name=column, # Nama untuk tooltip
            nbinsx=20,   
            marker_color=color,
            opacity=0.75,
            showlegend=False
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text="Distribusi Variabel Penelitian (2021-2023)",
    title_x=0.5,
    height=300 * n_rows,
    bargap=0.1           # Jarak antar batang
)

# Tampilkan
fig.show()

# Simpan sebagai HTML
fig.write_html(os.path.join(path_visualisasi, "histogram.html"))
print("--> File tersimpan: histogram.html")

Membuat Histogram Distribusi Data...


--> File tersimpan: histogram.html


### **6. Matriks Korelasi**

In [31]:
print("Membuat Heatmap Korelasi...")

corr_matrix = df_numeric.corr()

fig = px.imshow(corr_matrix,
                text_auto='.2f', # Tampilkan angka
                aspect="auto",
                color_continuous_scale='RdBu_r', # Skala warna Merah-Biru
                zmin=-1, zmax=1,
                title="Matriks Korelasi Antar Variabel (2021-2023)"
                )

fig.update_layout(
    title_x=0.5, # Judul di tengah
    width=1000,   # Lebar gambar
    height=900,  # Tinggi gambar
    xaxis_title="Variabel",
    yaxis_title="Variabel"
)

# 4. Tampilkan
fig.show()

# 5. Simpan ke HTML
# output_file = os.path.join(path_visualisasi, "heatmap.html")
# fig.write_html(output_file)
# print(f"--> File tersimpan: {output_file}")

print("\nDeteksi Multikolinearitas (Korelasi > 0.8)")
pairs = corr_matrix.abs().unstack().sort_values(ascending=False)
high_corr_pairs = pairs[(pairs > 0.8) & (pairs < 1.0)]

if not high_corr_pairs.empty:
    print("[PERINGATAN] Ditemukan pasangan variabel dengan korelasi sangat tinggi:")
    print(high_corr_pairs.drop_duplicates())
else:
    print("Aman: Tidak ditemukan korelasi antar variabel independen di atas 0.8.")

Membuat Heatmap Korelasi...



Deteksi Multikolinearitas (Korelasi > 0.8)
[PERINGATAN] Ditemukan pasangan variabel dengan korelasi sangat tinggi:
HLS          RLS   0.91
Pengeluaran  RLS   0.82
dtype: float64


### **7. Menyiapkan Data Peta**

In [ ]:
print("Menyiapkan data geospasial...")

numeric_cols = ['P1', 'UHH', 'HLS', 'RLS', 'Pengeluaran', 'TPT', 'Kepadatan']

# 1. Hitung Rata-rata 3 Tahun (2021-2023)
# Kita gunakan groupby untuk mendapatkan satu nilai rata-rata per wilayah
df_avg = df.groupby('Wilayah')[numeric_cols].mean().reset_index()
print(f"Rata-rata dihitung untuk {len(df_avg)} wilayah.")

# 2. Load Shapefile
# Pastikan file .shp, .shx, .dbf ada di folder 'maps/'
try:
    shapefile_path = os.path.join(path_maps, "Geology jawa Tengah.shp")
    gdf = gpd.read_file(shapefile_path)
    print(f"Shapefile berhasil dimuat: {len(gdf)} baris (poligon).")
except Exception as e:
    print(f"ERROR: Gagal memuat shapefile. {e}")
    # Stop jika shapefile gagal load, karena tidak bisa lanjut map
    raise

# 3. Standarisasi Nama Wilayah untuk Merge (PENTING!)
# Masalah umum: Di data BPS namanya "3301 Kabupaten Cilacap", di Peta namanya "CILACAP".
# Kita harus samakan keduanya agar bisa 'kawin' (merge).

# Bersihkan nama di Data BPS
df_avg['wilayah_join'] = df_avg['Wilayah'].astype(str).str.replace(r'^\d+\s+', '', regex=True) # Hapus kode angka depan
df_avg['wilayah_join'] = df_avg['wilayah_join'].str.replace('Kabupaten ', '').str.replace('Kota ', '').str.strip().str.upper()

# Bersihkan nama di Shapefile
# Kita cari kolom nama kabupaten di shapefile. Biasanya 'KABKOT' atau 'WADMKC'.
print("Kolom yang ada di Shapefile:", gdf.columns.tolist())

target_col_shp = 'NAME' # <--- GANTI INI jika nama kolom di shapefile kamu beda

if target_col_shp in gdf.columns:
    gdf['wilayah_join'] = gdf[target_col_shp].astype(str).str.upper().str.strip()
    
    # 4. Lakukan Merge
    # Kita tempelkan data angka ke peta
    gdf_map = gdf.merge(df_avg, on='wilayah_join', how='left')
    
    print(f"Merge selesai. Total wilayah di peta akhir: {len(gdf_map)}")
    
    # 5. Cek Validasi Merge (Sangat Penting)
    # Kita cek apakah ada wilayah di Data BPS yang GAGAL masuk ke Peta
    unmatched = df_avg[~df_avg['wilayah_join'].isin(gdf['wilayah_join'])]
    
    if not unmatched.empty:
        print("\n⚠️ PERINGATAN: Ada wilayah yang TIDAK COCOK namanya dengan Shapefile:")
        print(unmatched['wilayah_join'].tolist())
        print("Tips: Cek ejaan di shapefile (misal: 'MAGELANG' vs 'KOTA MAGELANG') dan sesuaikan kodenya.")
    else:
        print("\n✅ SUKSES: Semua data wilayah berhasil masuk ke peta!")

else:
    print(f"\n❌ ERROR: Kolom '{target_col_shp}' tidak ditemukan di Shapefile.")
    print("Silakan cek output 'Kolom yang ada di Shapefile' di atas, lalu ganti variabel 'target_col_shp' di kode ini.")

Menyiapkan data geospasial...
Rata-rata dihitung untuk 35 wilayah.
Shapefile berhasil dimuat: 335 baris (poligon).
Kolom yang ada di Shapefile: ['SYMBOLS', 'NM_LEMBAR', 'NO_LEMBAR', 'PROJECT', 'NAME', 'FORMATION', 'CLASS_LITH', 'T_CLASS_EN', 'B_CLASS_EN', 'DESCRIP1', 'Shape_Leng', 'Shape_Area', 'geometry']

❌ ERROR: Kolom 'Wilayah' tidak ditemukan di Shapefile.
Silakan cek output 'Kolom yang ada di Shapefile' di atas, lalu ganti variabel 'target_col_shp' di kode ini.
